In [59]:
from copy import deepcopy
import subprocess
import numpy as np
import open3d as o3d
import trimesh
import os
import time
import scipy
from networkx import radius

print(o3d.__version__)


0.18.0


In [3]:
dataset = 'Dancer'
GoF = 5

In [29]:
input_dir = os.path.join(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data', os.path.join(dataset, 'meshes'))

output_root_dir = os.path.join(r'G:\PycharmProjects\Mesh_Editing\Results\encode_Draco', dataset)
print(input_dir)
print(output_root_dir)
obj_files = [f for f in os.listdir(input_dir) if f.endswith('.obj')]

G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes
G:\PycharmProjects\Mesh_Editing\Results\encode_Draco\Dancer


In [42]:
dancer_5 = o3d.io.read_triangle_mesh(os.path.join(input_dir, obj_files[0]))
dancer_5.compute_vertex_normals()

dancer_5_pcd = o3d.geometry.PointCloud()
dancer_5_pcd.points = dancer_5.vertices


dancer_5_keypoints = o3d.geometry.keypoint.compute_iss_keypoints(dancer_5_pcd)


dancer_5.compute_vertex_normals()
dancer_5.paint_uniform_color([0.5, 0.5, 0.5])
dancer_5_keypoints.paint_uniform_color([1.0, 0.75, 0.0])
o3d.visualization.draw_geometries([dancer_5_keypoints, dancer_5])
print(dancer_5_keypoints)

PointCloud with 417 points.


In [40]:
dancer_6 = o3d.io.read_triangle_mesh(os.path.join(input_dir, obj_files[1]))
dancer_6.compute_vertex_normals()

dancer_6_pcd = o3d.geometry.PointCloud()
dancer_6_pcd.points = dancer_6.vertices
dancer_6_keypoints = o3d.geometry.keypoint.compute_iss_keypoints(dancer_6_pcd)
dancer_6.compute_vertex_normals()

dancer_6_keypoints.paint_uniform_color([1.0, 0.75, 0.0])
o3d.visualization.draw_geometries([dancer_6_keypoints, dancer_6])
print(dancer_6_keypoints)

PointCloud with 408 points.


In [ ]:
o3d.visualization.draw_geometries([dancer_5, dancer_6])

In [58]:
reg_p2p = o3d.pipelines.registration.registration_icp(dancer_5_keypoints, dancer_6_keypoints, 0.1)
print(reg_p2p.transformation)

[[ 9.99954662e-01 -9.27937280e-03 -2.13688974e-03  3.50598261e-04]
 [ 9.27143807e-03  9.99950197e-01 -3.69365908e-03 -2.47279284e-03]
 [ 2.17105816e-03  3.67367958e-03  9.99990895e-01 -3.71327693e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [50]:
def draw_registration_result(source, target, transformation):
    source_temp = deepcopy(source)
    target_temp = deepcopy(target)

    source_temp.transform(transformation)

    # This is patched version for tutorial rendering.
    # Use `draw` function for you application.
    o3d.visualization.draw_geometries([source_temp, target_temp])
dancer_6_keypoints.paint_uniform_color([1.0, 0, 0.0])
draw_registration_result(dancer_5_keypoints, dancer_6_keypoints, reg_p2p.transformation)

In [35]:
o3d.visualization.draw_geometries([dancer_5, dancer_6])

In [9]:
def compute_distance(mesh):
    edges = set()
    triangles = np.asarray(mesh.triangles)
    # Extract unique edges from the triangles
    for tri in triangles:
        for i in range(3):
            edge = tuple(sorted((tri[i], tri[(i + 1) % 3])))
            edges.add(edge)

    vertices = np.asarray(mesh.vertices)
    edge_lengths = []
    # Calculate the length of each edge
    for edge in edges:
        v1, v2 = edge
        length = np.linalg.norm(vertices[v1] - vertices[v2])
        edge_lengths.append(length)
    # Compute the average edge length
    average_length = np.mean(edge_lengths)
    radius = average_length * 6
    return radius

# get N(xl) and N(yt)
def find_neighbors_mesh_radius(mesh, vertex, radius):
    pcd = o3d.geometry.PointCloud()
    vertices = np.array(mesh.vertices)
    pcd.points = o3d.utility.Vector3dVector(vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    [k, idx, _] = pcd_tree.search_radius_vector_3d(vertex, radius)
    idx_np = np.asarray(idx[0:]) 
    neighbors = np.asarray(pcd.points)[idx_np]
    return neighbors #N(xl)

def find_neighbors_mesh_KNN(mesh, vertex, Q = 5):
    pcd = o3d.geometry.PointCloud()
    vertices = np.array(mesh.vertices)
    pcd.points = o3d.utility.Vector3dVector(vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    [k, idx, _] = pcd_tree.search_knn_vector_3d(vertex, Q)
    idx_np = np.asarray(idx[0:]) 
    neighbors = np.asarray(pcd.points)[idx_np]
    return neighbors #N(xl)

def embedded_deform_vertex(mesh, rotation, translation, keynodes):
    vertices = np.array(mesh.vertices)
    updated_vertices = deepcopy(vertices)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(keynodes)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    for i, xi in range(len(vertices)):
        [k, idx, _] = pcd_tree.search_knn_vector_3d(xi, 5)
        idx_np = np.asarray(idx[0:]) 
        neighbors = np.asarray(pcd.points)[idx_np]
        xi_updated = np.zeros(3)
        sigma = np.max(np.linalg.norm(neighbors - xi, axis=1))
        sum = 0
        for neighbor in neighbors:
            sum += np.square(1- np.linalg.norm(neighbor - xi)/sigma)
        for neighbor in neighbors:
            [k, idx, _] = pcd_tree.search_knn_vector_3d(neighbor, 1)
            s = idx[0]
            weight = np.square(1- np.linalg.norm(neighbor - xi)/sigma) / sum
            xi_updated += weight * (np.dot(rotation[s], (xi - neighbor)) + neighbor + translation[s])
        updated_vertices[i] = xi_updated
        
    mesh.vertices = o3d.utility.Vector3dVector(updated_vertices)
    return mesh

In [60]:
target = dancer_6
source = dancer_5

def function(params, source, target, keynodes):
    R_vec = params[3:]
    T = params[:3]
    R = R_vec.reshape(-1, 3)
    target_vertices = np.array(target.vertices)
    source_vertices = np.array(source.vertices)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(target_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    
    keynodespcd = o3d.geometry.PointCloud()
    keynodespcd.points = o3d.utility.Vector3dVector(keynodes)
    keynodespcd_tree = o3d.geometry.KDTreeFlann(keynodespcd)
    
    Error_motion = 0
    Error_rotation = 0
    Error_Regulation = 0
    
    for vertex in source_vertices:
        [k, idx, _] = pcd_tree.search_knn_vector_3d(vertex, 1)
        Error_motion += np.square(np.linalg.norm(vertex, target_vertices[idx[0]]))
    
    for i, keynode in keynodes:
        a1 = R[i][:, 0] 
        a2 = R[i][:, 1] 
        a3 = R[i][:, 2]
        term1 = (np.dot(a1, a2)) ** 2
        term2 = (np.dot(a1, a3)) ** 2
        term3 = (np.dot(a2, a3)) ** 2
        
        term4 = (1 - np.dot(a1, a1)) ** 2
        term5 = (1 - np.dot(a2, a2)) ** 2
        term6 = (1 - np.dot(a3, a3)) ** 2
        
        total_sum = term1 + term2 + term3 + term4 + term5 + term6
        Error_rotation += total_sum
        
    for i, keynode in keynodes:
        [k, idx, _] = keynodespcd_tree.search_knn_vector_3d(keynode, 5)
        idx_np = np.asarray(idx[0:]) 
        neighbors = np.asarray(keynodespcd.points)[idx_np]
        sigma = np.max(np.linalg.norm(neighbors - keynode, axis=1))
        sum = 0
        for neighbor in neighbors:
            sum += np.square(1- np.linalg.norm(neighbor - keynode)/sigma)
            
        
        for neighbor in neighbors:
            [k, idx, _] = keynodespcd_tree.search_knn_vector_3d(neighbor, 1)
            s = idx[0]
            weight = np.square(1- np.linalg.norm(neighbor - keynode)/sigma) / sum
            Error_Regulation += weight * (np.dot(R[s], (keynode - neighbor)) + neighbor + T[s])
    
    error  = 100 * Error_motion + 10 * Error_rotation + Error_Regulation
    return error